In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import norm ,skew,kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer,mean_squared_error


In [2]:
df_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv')

In [3]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288007 entries, 0 to 2288006
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(13), int64(95), object(1)
memory usage: 2.0 GB


In [4]:
c_int=df_train.select_dtypes(include=['int64']).apply(pd.to_numeric,downcast='unsigned')
c_float=df_train.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')

In [5]:
o_df=pd.concat([c_int,c_float],axis=1)
column_type=dict(o_df.dtypes.copy())
del o_df,c_int,c_float,df_train

In [6]:
data_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

data_test=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [7]:
data_train.rename(columns={'treatment_acceleraion_add_value':'treatment_acceleration_add_value',\
                          'sr_rss_a_prod_levell':'sr_rss_a_prod_level'} ,inplace=True)

data_test.rename(columns={'treatment_acceleraion_add_value':'treatment_acceleration_add_value',\
                          'sr_rss_a_prod_levell':'sr_rss_a_prod_level'} ,inplace=True)

In [8]:
data_train.shape

(2288007, 109)

In [9]:
data_train.head()

,user_id,register_time,wood_add_value,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,...,pvp_battle_count,pvp_lanch_count,pvp_win_count,pve_battle_count,pve_lanch_count,pve_win_count,avg_online_minutes,pay_price,pay_count,prediction_pay_price
0,1,2018-02-02 19:47:15,20125.0,3700.0,0.0,0.0,0.0,0.0,16375.0,2000.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
1,1593,2018-01-26 00:01:05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.333333,0.0,0,0.0
2,1594,2018-01-26 00:01:58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1.166667,0.0,0,0.0
3,1595,2018-01-26 00:02:13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,3.166667,0.0,0,0.0
4,1596,2018-01-26 00:02:46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,2.333333,0.0,0,0.0


In [10]:
train_user=data_train.iloc[:,:2]
train_data=data_train.iloc[:,2:]

In [11]:
train_user.shape

(2288007, 2)

In [12]:
train_data.shape

(2288007, 107)

In [13]:
column_name=train_data.columns.values[:32].tolist()
for i in range(len(column_name)):
    if (i+2) % 2 == 0 :
        name=column_name[i].replace('value','rate')
        #print(name)
        train_data[name]=train_data[column_name[i]] / (train_data[column_name[i+1]]+1)

In [14]:
train_data.shape

(2288007, 123)

In [15]:
train_data.iloc[:5,106:]

,prediction_pay_price,wood_add_rate,stone_add_rate,ivory_add_rate,meat_add_rate,magic_add_rate,infantry_add_rate,cavalry_add_rate,shaman_add_rate,wound_infantry_add_rate,wound_cavalry_add_rate,wound_shaman_add_rate,general_acceleration_add_rate,building_acceleration_add_rate,reaserch_acceleration_add_rate,training_acceleration_add_rate,treatment_acceleration_add_rate
0,0.0,5.437719,0.0,0.0,8.183409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,50.0,0.0
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
train_data['pvp_lanch_battle_rate']=train_data['pvp_battle_count']/(train_data['pvp_lanch_count']+1)
train_data['pvp_win_battle_rate']=train_data['pvp_battle_count']/(train_data['pvp_win_count']+1)
train_data['pvp_win_lanch_rate']=train_data['pvp_lanch_count']/(train_data['pvp_win_count']+1)
train_data['pve_win_battle_rate']=train_data['pvp_battle_count']/(train_data['pve_win_count']+1)
train_data['pay_price_rate']=train_data['pay_count']/(train_data['pay_price']+1)
train_data['price_online_minutes_rate']=train_data['avg_online_minutes']/(train_data['pay_price']+1)

In [17]:
train_data.columns.values[0:9]

array(['wood_add_value', 'wood_reduce_value', 'stone_add_value',
       'stone_reduce_value', 'ivory_add_value', 'ivory_reduce_value',
       'meat_add_value', 'meat_reduce_value', 'magic_add_value'],
      dtype=object)

In [18]:
train_data['resource_add_mean']=train_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.mean(),axis=1)
train_data['resource_add_med']=train_data.iloc[:,[0,2,4,6,8]].apply(lambda x:x.median(),axis=1)
train_data['resource_reduce_mean']=train_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.mean(),axis=1)
train_data['resource_reduce_med']=train_data.iloc[:,[1, 3, 5, 7, 9]].apply(lambda x:x.median(),axis=1)

train_data['resource_mean_rate']=train_data['resource_reduce_mean']/(train_data['resource_add_mean']+1)
train_data['resource_med_rate']=train_data['resource_reduce_med']/(train_data['resource_add_med']+1)

train_data['army_add_mean']=train_data.iloc[:,[10, 12, 14]].apply(lambda x:x.mean(),axis=1)
train_data['army_add_med']=train_data.iloc[:,[10, 12, 14]].apply(lambda x:x.median(),axis=1)
train_data['army_reduce_mean']=train_data.iloc[:,[11, 13, 15]].apply(lambda x:x.mean(),axis=1)
train_data['army_reduce_med']=train_data.iloc[:,[11, 13, 15]].apply(lambda x:x.median(),axis=1)

train_data['army_mean_rate']=train_data['army_reduce_mean']/(train_data['army_add_mean']+1)
train_data['army_med_rate']=train_data['army_reduce_med']/(train_data['army_add_med']+1)

train_data['wound_add_mean'] = train_data.iloc[:, [16, 18, 20]].apply(lambda x: x.mean(), axis=1)
train_data['wound_add_med'] = train_data.iloc[:, [16, 18, 20]].apply(lambda x: x.median(), axis=1)
train_data['wound_reduce_mean'] = train_data.iloc[:, [17, 19, 21]].apply(lambda x: x.mean(), axis=1)
train_data['wound_reduce_med'] = train_data.iloc[:, [17, 19, 21]].apply(lambda x: x.median(), axis=1)

train_data['wound_mean_rate']=train_data['wound_reduce_mean']/(train_data['wound_add_mean']+1)
train_data['wound_med_rate']=train_data['wound_reduce_med']/(train_data['wound_add_med']+1)

train_data['accelerate_add_mean'] = train_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.mean(), axis=1)
train_data['accelerate_add_med'] = train_data.iloc[:, [22, 24, 26, 28, 30]].apply(lambda x: x.median(), axis=1)
train_data['accelerate_reduce_mean'] = train_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.mean(), axis=1)
train_data['accelerate_reduce_med'] = train_data.iloc[:, [23, 25, 27, 29, 31]].apply(lambda x: x.median(), axis=1)

train_data['accelerate_mean_rate']=train_data['accelerate_reduce_mean']/(train_data['accelerate_add_mean']+1)
train_data['accelerate_med_rate']=train_data['accelerate_reduce_med']/(train_data['accelerate_add_med']+1)

In [26]:
train_user.register_time.head().apply(lambda x:str(x)).str[5:7]

0    02
1    01
2    01
3    01
4    01
Name: register_time, dtype: object

In [27]:
train_user['register_time_month']=pd.to_numeric(train_user.register_time.apply(lambda x:str(x)).str[5:7])
train_user['register_time_day']=pd.to_numeric(train_user.register_time.apply(lambda x:str(x)).str[8:10])
train_user.drop(labels=['register_time'],axis=1,inplace=True)
train_user['register_time_count']=train_user['register_time_month']*31+train_user['register_time_day']

In [28]:
train_user.head()

,user_id,register_time_month,register_time_day,register_time_count
0,1,2,2,64
1,1593,1,26,57
2,1594,1,26,57
3,1595,1,26,57
4,1596,1,26,57


In [29]:
train_data['register_time_month']=train_user['register_time_month']
train_data['register_time_day']=train_user['register_time_day']
train_data['register_time_count']=train_user['register_time_count']

In [30]:
train = train_data[train_data['pay_price'] > 0]

In [31]:
train.shape

(41439, 156)

In [32]:
train['new_pay_label']=train['prediction_pay_price']!=train['pay_price']
train['new_pay_label']=train['new_pay_label'].map({True:1,False:0})
train['new_pay_price']=train['prediction_pay_price']-train['pay_price']
train['new_pay_rate']=(train['prediction_pay_price']-train['pay_price'])/train['pay_price']

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [33]:
train['new_pay_label'].head()

25     0
40     1
47     0
86     0
119    0
Name: new_pay_label, dtype: int64

In [34]:
classify_train=train.iloc[:,1:-3]
classify_train.drop('prediction_pay_price',axis=1,inplace=True)

In [35]:
classify_train.duplicated().sum()

196

In [36]:
classify_train.head()

,wood_reduce_value,stone_add_value,stone_reduce_value,ivory_add_value,ivory_reduce_value,meat_add_value,meat_reduce_value,magic_add_value,magic_reduce_value,infantry_add_value,...,wound_med_rate,accelerate_add_mean,accelerate_add_med,accelerate_reduce_mean,accelerate_reduce_med,accelerate_mean_rate,accelerate_med_rate,register_time_month,register_time_day,register_time_count
25,97200.0,20000.0,0.0,0.0,0.0,160500.0,80392.0,12000.0,0.0,0,...,0.0,100.0,75.0,0.0,0.0,0.000000,0.000000,1,26,57
40,1137687.0,491331.0,790208.0,77000.0,0.0,1457249.0,708483.0,217455.0,162987.0,370,...,0.0,631.2,618.0,322.8,500.0,0.510598,0.807754,1,26,57
47,173248.0,70841.0,117012.0,55000.0,0.0,206366.0,82205.0,29902.0,88064.0,0,...,0.0,226.8,45.0,415.8,0.0,1.825285,0.000000,1,26,57
86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57
119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1,26,57


In [38]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.externals import joblib
model = GridSearchCV(
    estimator=XGBClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.1],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.8],
        'scale_pos_weight': [2.5, ],
        'min_child_weight': [2, ]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1)



cl_t = classify_train.iloc[:, :-3].drop_duplicates()
model.fit(cl_t, train.loc[cl_t.index,'new_pay_label'])
joblib.dump(model,'F:\\study\\ml\\DataSet\\Tap4fun\\Model\\xgb_clf.model')
print(model.best_score_, model.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

0.6433732698025395 {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 1000, 'scale_pos_weight': 2.5, 'subsample': 1}


In [39]:
model = GridSearchCV(
    estimator=XGBClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.1],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.8],
        'scale_pos_weight': [2.5, ],
        'min_child_weight': [2, ]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1)

cl_t = classify_train.iloc[:, :-3].drop_duplicates()
model.fit(cl_t, train.loc[cl_t.index,'new_pay_label'])
joblib.dump(model,'F:\\study\\ml\\DataSet\\Tap4fun\\Model\\xgb_clf.model')
print(model.best_score_, model.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:  1.3min finished


0.6433732698025395 {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 2, 'n_estimators': 1000, 'scale_pos_weight': 2.5, 'subsample': 1}


In [40]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.cross_validation import train_test_split,cross_val_score

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
X_train,X_test,y_train,y_test=train_test_split(cl_t,train.loc[cl_t.index,'new_pay_label'],test_size=0.2,random_state=13)
y_pred=model.predict(X_test)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [42]:
predictions=y_pred
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))


准确率 0.7954239569313594
混淆矩阵 [[4626 1252]
 [ 420 1875]]
分类报告              precision    recall  f1-score   support

          0       0.92      0.79      0.85      5878
          1       0.60      0.82      0.69      2295

avg / total       0.83      0.80      0.80      8173



In [44]:
model2=XGBClassifier( max_bin=128,n_estimators=1000,learning_rate=0.1,
                     max_depth=2,subsample=1,colsample_bytree=0.8,
                     scale_pos_weight=2.5,min_child_weight=2,scoring='f1'
                     ,n_jobs=4).fit(X_train,y_train)

In [45]:
predictions=model2.predict(X_test)
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))


准确率 0.7725437415881561
混淆矩阵 [[4545 1333]
 [ 526 1769]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.77      0.83      5878
          1       0.57      0.77      0.66      2295

avg / total       0.80      0.77      0.78      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [60]:
############### test lgb use same params
model10 = GridSearchCV(
    estimator=lgb.LGBMClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.1],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.8],
        'scale_pos_weight': [2.5, ],
        'min_child_weight': [2, ]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1)

cl_t = classify_train.iloc[:, :-3].drop_duplicates()
model10.fit(cl_t, train.loc[cl_t.index,'new_pay_label'])


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   30.7s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_bin=128, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [1000], 'learning_rate': [0.1], 'max_depth': [2], 'subsample': [1], 'colsample_bytree': [0.8], 'scale_pos_weight': [2.5], 'min_child_weight': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=1)

In [61]:
predictions=model10.predict(X_test)
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.7930992291692157
混淆矩阵 [[4620 1258]
 [ 433 1862]]
分类报告              precision    recall  f1-score   support

          0       0.91      0.79      0.85      5878
          1       0.60      0.81      0.69      2295

avg / total       0.83      0.79      0.80      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
###############################################



In [46]:
df_imp=pd.DataFrame()
df_imp['name']=X_train.columns.values
df_imp['socre']=model2.feature_importances_


In [47]:
df_imp.sort_values(by=['socre'],ascending=False)

,name,socre
126,price_online_minutes_rate,0.035076
102,avg_online_minutes,0.025295
109,magic_add_rate,0.024958
116,general_acceleration_add_rate,0.024283
117,building_acceleration_add_rate,0.023946
118,reaserch_acceleration_add_rate,0.022597
3,ivory_add_value,0.019562
119,training_acceleration_add_rate,0.018887
106,stone_add_rate,0.018550
108,meat_add_rate,0.017538


In [48]:
import lightgbm as lgb

In [49]:
data_train=lgb.Dataset(X_train,y_train)
# param = { 'boosting_type':'gbdt','num_leaves':200, 'objective':'binary','max_depth':8,'learning_rate':.05,'max_bin':1000,
#              'nthread':4, 'silent':True
#         }

param = { 'boosting_type':'gbdt','objective':'binary',
             'nthread':4, 'silent':True
        }
param['metric'] = ['auc', 'binary_logloss']
num_round=1000
lgbm=lgb.train(param,data_train,num_round,verbose_eval=50)

D:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:661: UserWarning: silent keyword has been found in `params` and will be ignored. Please use silent argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


In [50]:
ypred2=lgbm.predict(X_test)
y_pred_proba=ypred2
thresholds = [0.1,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.6,0.65,0.7,0.75,0.8,0.85,0.9]
for i in thresholds:
    y_test_predictions_high_recall =y_pred_proba > i
    print('Threshold >= %s'%i)
    print('准确率',accuracy_score(y_test,y_test_predictions_high_recall))
    print (u'混淆矩阵',confusion_matrix(y_test,y_test_predictions_high_recall))
    print (u'分类报告',classification_report(y_test,y_test_predictions_high_recall))

Threshold >= 0.1
准确率 0.6947265386027163
混淆矩阵 [[3650 2228]
 [ 267 2028]]
分类报告              precision    recall  f1-score   support

          0       0.93      0.62      0.75      5878
          1       0.48      0.88      0.62      2295

avg / total       0.80      0.69      0.71      8173

Threshold >= 0.2
准确率 0.7567600636241282
混淆矩阵 [[4397 1481]
 [ 507 1788]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.75      0.82      5878
          1       0.55      0.78      0.64      2295

avg / total       0.80      0.76      0.77      8173

Threshold >= 0.25
准确率 0.7715649088462009
混淆矩阵 [[4628 1250]
 [ 617 1678]]
分类报告              precision    recall  f1-score   support

          0       0.88      0.79      0.83      5878
          1       0.57      0.73      0.64      2295

avg / total       0.80      0.77      0.78      8173

Threshold >= 0.3
准确率 0.7836779640278967
混淆矩阵 [[4826 1052]
 [ 716 1579]]
分类报告              precision    recall  f1-score   s

In [59]:
df_fi=pd.DataFrame()
df_fi['name']=lgbm.feature_name()
df_fi['importance']=lgbm.feature_importance()
df_fi.sort_values(by=['importance'],axis=0,ascending=False)
# df_fi.sort_values(by=['importance'])


,name,importance
102,avg_online_minutes,719
109,magic_add_rate,623
117,building_acceleration_add_rate,577
118,reaserch_acceleration_add_rate,553
116,general_acceleration_add_rate,534
126,price_online_minutes_rate,533
105,wood_add_rate,525
108,meat_add_rate,510
29,treatment_acceleration_add_value,508
27,training_acceleration_add_value,501


In [126]:
model3=XGBClassifier( scoring='f1',n_jobs=4).fit(X_train,y_train)
ypred3=model3.predict(X_test)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [128]:
predictions=ypred3
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.809005261225988
混淆矩阵 [[5372  506]
 [1055 1240]]
分类报告              precision    recall  f1-score   support

          0       0.84      0.91      0.87      5878
          1       0.71      0.54      0.61      2295

avg / total       0.80      0.81      0.80      8173



In [77]:
model4 = GridSearchCV(
    estimator=lgb.LGBMClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.1],
        'max_depth': [5],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   30.4s finished


0.6417552374297396
{'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1000, 'scale_pos_weight': 2.5, 'subsample': 0.8}


In [78]:
ypred4=model4.predict(X_test)
predictions=ypred4
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.8038663893307231
混淆矩阵 [[5306  572]
 [1031 1264]]
分类报告              precision    recall  f1-score   support

          0       0.84      0.90      0.87      5878
          1       0.69      0.55      0.61      2295

avg / total       0.80      0.80      0.80      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [79]:
model41= GridSearchCV(
    estimator=lgb.LGBMClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.01,0.05,0.1],
        'max_depth': [2,4],
        'subsample': [0.8,0.9],
        'colsample_bytree': [0.8,0.9],
        'scale_pos_weight': [2.5,2.6]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 144 out of 144 | elapsed: 10.3min finished


In [82]:
print(model41.best_score_)
print(model41.best_params_)

0.6417552374297396
{'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 1000, 'scale_pos_weight': 2.5, 'subsample': 0.8}


In [80]:
ypred41=model41.predict(X_test)
predictions=ypred41
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.7702190138260124
混淆矩阵 [[4497 1381]
 [ 497 1798]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.77      0.83      5878
          1       0.57      0.78      0.66      2295

avg / total       0.81      0.77      0.78      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [89]:
model42= GridSearchCV(
    estimator=lgb.LGBMClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.01],
        'max_depth': [4],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
       # 'scale_pos_weight': [2.6],
        'is_unbalance':[True]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   18.3s finished


In [90]:
ypred42=model41.predict(X_test)
predictions=ypred42
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

准确率 0.7702190138260124
混淆矩阵 [[4497 1381]
 [ 497 1798]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.77      0.83      5878
          1       0.57      0.78      0.66      2295

avg / total       0.81      0.77      0.78      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [86]:
y_test[y_test.values==0].count() / y_test[y_test.values ==1].count()

2.561220043572985

In [54]:
#################
# 经过以上测试,无论 lgb还是xgb,在默认参数下,都不能取得较好的效果
#故准备hyperoter搜索最佳参数

In [55]:
import time
from hyperopt import hp,fmin,rand,tpe,space_eval,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
lgb.Lg

In [92]:
def objective(params):
    params={
        'num_leaves':int(params['num_leaves']),
        'colsample_bytree':'{:.3f}'.format(params['colsample_bytree']),
        'max_depth':int(params['max_depth']),
        'learning_rate':'{:.3f}'.format(params['learning_rate']),
        'bagging_fraction':'{:.1f}'.format(params['bagging_fraction']),
        'colsample_bytree':'{:.2f}'.format(params['colsample_bytree']),
         'lambda':int(params['lambda']),
         'min_child_weight':int(params['min_child_weight']),
        
    }
    clf=lgb.LGBMClassifier(
    n_estimators=1000,
    is_unbalance=True,
    **params
    )
    
    score=cross_val_score(clf,X_train,y_train,scoring='f1',cv=StratifiedKFold()).mean()
    print('best {:.3f} params {}'.format(score,params))
    return score

space={
    'num_leaves':hp.quniform('num_leaves',8,128,2),
    'colsample_bytree':hp.uniform('colsample_bytree',0.3,1),
    'max_depth':hp.quniform('max_depth',2,7,1),
	'learning_rate':hp.uniform('learning_rate',0.01,0.5),
	'bagging_fraction':hp.uniform('bagging_fraction',0.3,1),
	'colsample_bytree':hp.uniform('feature_fraction',0.3,1),
    'lambda':hp.uniform('lambda',1,50),
    'min_child_weight':hp.quniform('min_child_weight',1,10,1)
}


best=fmin(fn=objective,
         space=space,
          algo=tpe.suggest,
          max_evals=1000
         )

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best 0.603 params {'num_leaves': 96, 'colsample_bytree': '0.97', 'max_depth': 3, 'learning_rate': '0.367', 'bagging_fraction': '0.9', 'lambda': 9, 'min_child_weight': 10}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best 0.615 params {'num_leaves': 16, 'colsample_bytree': '0.81', 'max_depth': 3, 'learning_rate': '0.254', 'bagging_fraction': '0.7', 'lambda': 6, 'min_child_weight': 7}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best 0.593 params {'num_leaves': 122, 'colsample_bytree': '0.50', 'max_depth': 5, 'learning_rate': '0.456', 'bagging_fraction': '0.6', 'lambda': 8, 'min_child_weight': 2}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best 0.588 params {'num_leaves': 110, 'colsample_bytree': '0.61', 'max_depth': 3, 'learning_rate': '0.500', 'bagging_fraction': '0.9', 'lambda': 14, 'min_child_weight': 1}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [93]:
print("Hyperopt estimated optimum {}".format(best))

Hyperopt estimated optimum {'bagging_fraction': 0.6311310785956743, 'feature_fraction': 0.4529670683544176, 'lambda': 1.989600277131583, 'learning_rate': 0.4797678795231437, 'max_depth': 4.0, 'min_child_weight': 4.0, 'num_leaves': 52.0}


In [75]:
y_train[y_train.values==1].count()

9009

In [74]:
y_train[y_train.values==0].count()

23681

In [76]:
y_train[y_train.values==0].count()/y_train[y_train.values==1].count()

2.6285936285936287

In [98]:
def objective2(params):
    params={
        'num_leaves':int(params['num_leaves']),
        'colsample_bytree':'{:.3f}'.format(params['colsample_bytree']),
        'max_depth':int(params['max_depth']),
        'learning_rate':'{:.3f}'.format(params['learning_rate']),
        'bagging_fraction':'{:.1f}'.format(params['bagging_fraction']),
        'colsample_bytree':'{:.2f}'.format(params['colsample_bytree']),
         'lambda':int(params['lambda']),
         'min_child_weight':int(params['min_child_weight']),
        
    }
    clf=lgb.LGBMClassifier(
    n_estimators=100,
    is_unbalance=True,
    **params
    )
    
    score=-cross_val_score(clf,X_train,y_train,scoring='f1',cv=StratifiedKFold()).mean()
    print('best {:.3f} params {}'.format(score,params))
    return score

space2={
    'num_leaves':hp.quniform('num_leaves',8,128,2),
    'colsample_bytree':hp.uniform('colsample_bytree',0.3,1),
    'max_depth':hp.quniform('max_depth',2,7,1),
	'learning_rate':hp.uniform('learning_rate',0.01,0.5),
	'bagging_fraction':hp.uniform('bagging_fraction',0.3,1),
	'colsample_bytree':hp.uniform('feature_fraction',0.3,1),
    'lambda':hp.uniform('lambda',1,50),
    'min_child_weight':hp.quniform('min_child_weight',1,10,1)
}


best2=fmin(fn=objective2,
         space=space2,
          algo=tpe.suggest,
          max_evals=50
         )

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best -0.632 params {'num_leaves': 92, 'colsample_bytree': '0.75', 'max_depth': 5, 'learning_rate': '0.209', 'bagging_fraction': '0.6', 'lambda': 4, 'min_child_weight': 5}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best -0.634 params {'num_leaves': 94, 'colsample_bytree': '0.30', 'max_depth': 2, 'learning_rate': '0.105', 'bagging_fraction': '0.8', 'lambda': 4, 'min_child_weight': 4}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best -0.614 params {'num_leaves': 108, 'colsample_bytree': '0.95', 'max_depth': 5, 'learning_rate': '0.419', 'bagging_fraction': '0.5', 'lambda': 42, 'min_child_weight': 3}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


best -0.611 params {'num_leaves': 100, 'colsample_bytree': '0.55', 'max_depth': 6, 'learning_rate': '0.328', 'bagging_fraction': '0.9', 'lambda': 16, 'min_child_weight': 5}


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [99]:
best

{'bagging_fraction': 0.6311310785956743,
 'feature_fraction': 0.4529670683544176,
 'lambda': 1.989600277131583,
 'learning_rate': 0.4797678795231437,
 'max_depth': 4.0,
 'min_child_weight': 4.0,
 'num_leaves': 52.0}

In [ ]:
##############################

In [100]:
model43= GridSearchCV(
    estimator=lgb.LGBMClassifier( max_bin=128),
    param_grid={
        'n_estimators': [1000],
        'learning_rate': [0.48],
        'max_depth': [4],
        'subsample': [0.65],
        'colsample_bytree': [0.45],
       # 'scale_pos_weight': [2.6],
        'is_unbalance':[True]
    },
    scoring='f1',
    cv=3,
    n_jobs=4,
    verbose=1).fit(X_train,y_train)

ypred43=model41.predict(X_test)
predictions=ypred43
print('准确率',accuracy_score(y_test,predictions))
print('混淆矩阵',confusion_matrix(y_test,predictions))
print('分类报告',classification_report(y_test,predictions))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   15.2s finished


准确率 0.7702190138260124
混淆矩阵 [[4497 1381]
 [ 497 1798]]
分类报告              precision    recall  f1-score   support

          0       0.90      0.77      0.83      5878
          1       0.57      0.78      0.66      2295

avg / total       0.81      0.77      0.78      8173



D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
######################   for regression

In [101]:
classify_train.shape

(41439, 154)

In [102]:
X_train.shape

(32690, 151)

In [103]:
X_train.iloc[:5,[105, 126, 121, 132, 150]]

,wood_add_rate,price_online_minutes_rate,pvp_lanch_battle_rate,resource_med_rate,accelerate_med_rate
1345843,1.050263,21.121012,1.333333,0.967158,0.971725
2090091,1.155318,27.483284,1.375000,0.794296,0.463165
1881402,1.013236,8.720518,1.036697,0.953199,0.999034
2256801,1.122143,87.520943,3.666667,0.767616,0.565968
66720,1.830980,5.480984,0.000000,0.001500,0.097222


In [119]:
lgbm2=lgb.LGBMClassifier(boosting_type='gbdt',
nthread=4, 
silent=True,
n_estimators= 1000,
learning_rate= 0.48,
max_depth= 4,
subsample= 0.65,
colsample_bytree= 0.45,
is_unbalance=True).fit(X_train,y_train)

In [123]:
df_fi=pd.DataFrame()
# df_fi['name']=lgbm2.feature_name()
df_fi['importance']=lgbm2.feature_importances_
df_fi.sort_values(by=['importance'],axis=0,ascending=False).head(10)


,importance
102,276
126,259
117,230
109,224
108,210
27,208
119,206
116,197
29,196
121,190


In [125]:
from xgboost import XGBRegressor

In [126]:
model_reg = GridSearchCV(
    estimator=XGBRegressor( max_bin=128),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=1,
    verbose=4)
model_reg.fit(classify_train.iloc[:, [105, 126, 121, 132, 150]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg.best_score_), model_reg.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
47.773954117508126 {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'reg_alpha': 13.4, 'subsample': 1}


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s finished


In [ ]:
classify_train.iloc[:, [105, 126, 121, 132, 150]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1]

In [128]:
model_reg2 = GridSearchCV(
    estimator=lgb.LGBMRegressor( max_bin=128),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=4,
    verbose=1)
model_reg2.fit(classify_train.iloc[:, [105, 126, 121, 132, 150]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg2.best_score_), model_reg2.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:   10.4s finished


47.542978965415244 {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'reg_alpha': 13.4, 'subsample': 1}


In [129]:
model_reg3 = GridSearchCV(
    estimator=lgb.LGBMRegressor( max_bin=128),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=4,
    verbose=1)
model_reg2.fit(classify_train.iloc[:, [102, 126, 117, 109, 108]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg2.best_score_), model_reg2.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
47.54216025076167 {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'reg_alpha': 13.4, 'subsample': 1}


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    2.4s finished


In [183]:
model_reg3 = GridSearchCV(
    estimator=lgb.LGBMRegressor( max_bin=128),
    #     estimator=LinearRegression(),
    param_grid={
        'n_estimators': [80],
        'learning_rate': [0.05],
        'max_depth': [2],
        'subsample': [1],
        'colsample_bytree': [0.5],
        'reg_alpha': [13.4],
    },
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=4,
    verbose=1)
model_reg3.fit(classify_train.iloc[:, [102, 126, 117, 109, 108]][train['new_pay_label'] == 1],
              train.iloc[:, -1][train['new_pay_label'] == 1])
# model_reg.fit(cl_t.iloc[:,[105,126,121,132,150]][train.loc[cl_t.index,'new_pay_label']==1],train.iloc[cl_t.index,-1][train.loc[cl_t.index,'new_pay_label']==1])
print(np.sqrt(-model_reg3.best_score_), model_reg3.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=4)]: Done   3 out of   3 | elapsed:    2.7s finished


47.54216025076167 {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 80, 'reg_alpha': 13.4, 'subsample': 1}
